In [ ]:
import cv2
import cvzone
from cvzone.HandTrackingModule import HandDetector
import numpy as np

# Initialize the hand detector
detector = HandDetector(detectionCon=0.8, maxHands=2)

# Load images
def load_images():
    imgBackground = cv2.imread(""Image-location path"/ based pong game/Background.jpg")
    imgGameOver = cv2.imread(""Image-location path"/Gestures based pong game/Background.jpg")
    imgBall = cv2.imread(""Image-location path"/Gestures based pong game/ball.jpg")
    imgBat1 = cv2.imread(""Image-location path"/Gestures based pong game/bat1.jpg", cv2.IMREAD_UNCHANGED)
    imgBat2 = cv2.imread(""Image-location path"/Gestures based pong game/bat2.jpg", cv2.IMREAD_UNCHANGED)

    if any(image is None for image in [imgBackground, imgGameOver, imgBall, imgBat1, imgBat2]):
        print("Error: One or more images failed to load.")
        exit()

    # Add alpha channel to bats
    imgBat1 = add_alpha_channel(imgBat1)
    imgBat2 = add_alpha_channel(imgBat2)

    return imgBackground, imgGameOver, imgBall, imgBat1, imgBat2

# Add alpha channel if not present
def add_alpha_channel(image):
    if image.shape[2] == 3:
        b, g, r = cv2.split(image)
        alpha = np.ones(b.shape, dtype=b.dtype) * 255  # Fully opaque
        return cv2.merge([b, g, r, alpha])
    return image

# Initialize the game state
def initialize_game():
    return [100, 100], 15, 15, False, [0, 0]  # ballPos, speedX, speedY, gameOver, score

def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Camera not accessible.")
        return

    cap.set(3, 1280)
    cap.set(4, 720)

    imgBackground, imgGameOver, imgBall, imgBat1, imgBat2 = load_images()
    ballPos, speedX, speedY, gameOver, score = initialize_game()

    while True:
        success, img = cap.read()
        if not success:
            break
        img = cv2.flip(img, 1)
        imgRaw = img.copy()

        hands, img = detector.findHands(img, flipType=False)
        imgBackground_resized = cv2.resize(imgBackground, (img.shape[1], img.shape[0]))
        img = cv2.addWeighted(img, 0.2, imgBackground_resized, 0.8, 0)

        if hands:
            for hand in hands:
                x, y, w, h = hand['bbox']
                h1, w1, _ = imgBat1.shape
                y1 = np.clip(y - h1 // 2, 20, img.shape[0] - h1 - 20)

                if hand['type'] == "Left":
                    img = cvzone.overlayPNG(img, imgBat1, (59, y1))
                    if 59 < ballPos[0] < 59 + w1 and y1 < ballPos[1] < y1 + h1:
                        speedX = -speedX
                        ballPos[0] += 30
                        score[0] += 1

                if hand['type'] == "Right":
                    img = cvzone.overlayPNG(img, imgBat2, (img.shape[1] - w1 - 59, y1))
                    if img.shape[1] - w1 - 59 < ballPos[0] < img.shape[1] - 59 and y1 < ballPos[1] < y1 + h1:
                        speedX = -speedX
                        ballPos[0] -= 30
                        score[1] += 1

        if ballPos[0] < 40 or ballPos[0] > img.shape[1] - 40:
            gameOver = True

        if not gameOver:
            if ballPos[1] >= img.shape[0] - 10 or ballPos[1] <= 10:
                speedY = -speedY

            ballPos[0] += speedX
            ballPos[1] += speedY

            ball_height, ball_width, _ = imgBall.shape
            ball_x1 = ballPos[0] - ball_width // 2
            ball_y1 = ballPos[1] - ball_height // 2

            if 0 <= ball_x1 < img.shape[1] - ball_width and 0 <= ball_y1 < img.shape[0] - ball_height:
                img[ball_y1:ball_y1 + ball_height, ball_x1:ball_x1 + ball_width] = imgBall

            cv2.putText(img, str(score[0]), (300, 650), cv2.FONT_HERSHEY_COMPLEX, 3, (255, 255, 255), 5)
            cv2.putText(img, str(score[1]), (900, 650), cv2.FONT_HERSHEY_COMPLEX, 3, (255, 255, 255), 5)
        else:
            img[0:imgGameOver.shape[0], 0:imgGameOver.shape[1]] = imgGameOver

            if score[0] > score[1]:
                winner_text = "Player 1 Wins!"
            elif score[1] > score[0]:
                winner_text = "Player 2 Wins!"
            else:
                winner_text = "It's a Draw!"

            cv2.putText(img, winner_text, (400, 350), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 5)
            cv2.putText(img, f"Final Score: {score[0]} - {score[1]}", (420, 450), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 0), 5)
            cv2.putText(img, "Press 'R' to Restart", (440, 550), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 5)

        resized_imgRaw = cv2.resize(imgRaw, (213, 120))
        img[580:580 + resized_imgRaw.shape[0], 20:20 + resized_imgRaw.shape[1]] = resized_imgRaw

        cv2.imshow("Pong Game", img)
        key = cv2.waitKey(1)

        if key == ord('r'):
            ballPos, speedX, speedY, gameOver, score = initialize_game()
        elif key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()